In [1]:
import pandas as pd
import numpy as np
from dateutil.parser import *
from datetime import *

In [2]:
DEFAULT = datetime(1,1,1)

Py Notes

- Data reshapping is awesome! http://pandas.pydata.org/pandas-docs/stable/reshaping.html

## Price relationship btwn organics and conventional crops


Notes:
- exempt farms are those that say they farm organic but do less thank $5k/year, so dont have be to be certified.
- cropland - areas used for the production of adapted crops for harvest
- rangeland - native vegetation land used for livestock
- pastureland - seed land used for livestock

Links:

!UNFORTUNATELY THE USDA PRICE DATA HASNT BEEN UPDATED SINCE 2012
- [USDA's ERS organic vs conventional price data](http://www.ers.usda.gov/data-products/organic-prices.aspx)
- [USDA's ERS Documentation for prices](http://www.ers.usda.gov/data-products/organic-prices/documentation.aspx)

The weekly reports are kept upt to date, but they're pdfs
- [USDA's AMS Organic Reports (with commentary](http://www.ers.usda.gov/data-products/organic-prices/documentation.aspx)

- [Resources for Organic and Transitioning Farmers](http://ofrf.org/policy/resources)

-[USDA Oraganic Insider newsletter archive](http://archive.constantcontact.com/fs054/1103777415326/archive/1108212629106.html)

https://www.agcensus.usda.gov/Publications/2012/Online_Resources/Organics/ORGANICS.pdf

https://www.ams.usda.gov/market-news/organic

https://www.ams.usda.gov/market-news/organic-market-news-reports

https://www.ams.usda.gov/market-news/custom-reports


- "The 2015 Certified Organic Survey publication will be released in September 2016." [here](https://www.nass.usda.gov/Surveys/Guide_to_NASS_Surveys/Organic_Production/index.php)

- [2014 Organic Survey](https://www.agcensus.usda.gov/Publications/2012/Online_Resources/Organics/)

USDA Risk Management Agency: [organics page](http://www.rma.usda.gov/news/currentissues/organics/)
- [Organic Pricing Methodologies for Barley, Corn,Cotton,Grain Sorghum, Rice, Soybeans, Sunflowers,and Wheat](http://www.rma.usda.gov/pubs/2015/organicpricingmethodology.pdf)
- [Summary of business for organic production](http://www.rma.usda.gov/data/sob/organic/2015organic.pdf)

[March 2016 report to congress](http://www.rma.usda.gov/pubs/2016/appropriations.pdf)
"After conducting research, it was determined that for some crops, the organic counterpart does 
not receive a premium price. For those crops, RMA has concluded that a premium organic price 
election is not warranted and has the potential to distort market behavior.  These crops are 
alfalfa seed, carambola trees, cigar filler tobacco, cigar wrapper tobacco, fire cured tobacco, forage seeding, lemon trees, lime trees, mango trees, and sugar beets."

- [Auction Cash Market for organics](http://www.mercaris.com/company#about)



In [3]:
%pwd

'/home/brett/Documents/JupyterNotebooks/AgResearch'

In [4]:
%cd /home/brett/Dropbox/AgResearch/

/home/brett/Dropbox/AgResearch


## 2014 organic survey table 551
Transitional Organic Land Operated by Non-Organic Farms

bullets:
- 688 non-organic farms are transitioning to grow organic products. 50,688 acres total
    - 24,550 acres cropland, 17,030 acres patureland, 9,108 acres rangeland
    
    
    

In [5]:
!head -n 10 corg_p551_t0561.csv

In [6]:
acresInTransition = pd.read_csv('corg_p551_t0561.csv',skiprows=6)

In [7]:
acresInTransition.head(10)

,561,h,Geographic Area,Farms,Total,Cropland,Pastureland,Rangeland
0,561,h,NaN,NaN,NaN,NaN,NaN,NaN
1,561,d,United States,688.0,50688,24550,17030,9108
2,561,d,NaN,NaN,NaN,NaN,NaN,NaN
3,561,d,Alabama,6.0,(D),17,(D),-
4,561,d,Alaska,1.0,(D),-,(D),-
5,561,d,Arkansas,2.0,(D),(D),-,(D)
6,561,d,California,57.0,2596,1971,582,43
7,561,d,Colorado,13.0,680,491,69,120
8,561,d,NaN,NaN,NaN,NaN,NaN,NaN
9,561,d,Connecticut,8.0,26,25,1,-


In [8]:
#drop nan rows
acresInTransition = acresInTransition[np.isfinite(acresInTransition['Farms'])]

In [9]:
#del col '561'
acresInTransition = acresInTransition.drop('561',1)

In [10]:
#del col 'h'
acresInTransition = acresInTransition.drop('h',1)

In [11]:
#rename columns
acresInTransition.columns = ['Area','farmsCount','totalAcresInTrans','croplandAcresInTrans',
                     'pasturelandAcresInTrans','rangelandAcresInTrans']

## 2014 organic survey table 1
Farms, land and value of sales of organic ag products - certified and exempt farms

bullets:

- There are 14,048 organic farms operating in the United states. The total acreage is 3,670,560 acres.
    - thats 261 acres per farm
- 1,365 farms are classified as transitional, with 122,175 acres transitioning. 
    - thats 89 acres per farm

In [12]:
!head -n 10 corg_p011_t0001.csv

In [13]:
farmsLandValue = pd.read_csv('corg_p011_t0001.csv',skiprows=4)

In [14]:
#drop nan rows
farmsLandValue = farmsLandValue[np.isfinite(farmsLandValue['United States'])]

In [15]:
#del cols
farmsLandValue = farmsLandValue.drop(farmsLandValue.columns[[0,1,5,6,7,8,9]],1)

In [16]:
#forward fill item names to get rid of nans
farmsLandValue['Item'] = farmsLandValue['Item'].fillna(method='ffill')

In [17]:
#rename columns
farmsLandValue.columns = ['Item','Type','UnitedStatesTotal']

In [18]:
farmsLandValue.head(10)

,Item,Type,UnitedStatesTotal
3,Certified or exempt,farms,14093.0
8,operated,farms,14048.0
9,operated,acres,3670560.0
11,Land owned,farms,12353.0
12,Land owned,acres,2221715.0
13,Land rented from others,farms,5582.0
14,Land rented from others,acres,1507957.0
15,Land rented to others,farms,521.0
16,Land rented to others,acres,59112.0
18,Total certified organic acres operated,farms,12595.0


## ERS organic price tables [old data]

### Wholesale Veggies

In [19]:
!head -n 5 Vegetables2012.csv

" Wholesale vegetable prices, organic and conventional, monthly and annual, 2012",,,,,,,,,,,,,,,,,
Commodity,Subgroup,Package,Organic/ Conventional,Terminal Market,Jan-12,Feb-12,Mar-12,Apr-12,May-12,Jun-12,Jul-12,Aug-12,Sep-12,Oct-12,Nov-12,Dec-12,2012
Artichoke (SF only),Globe,"cartons, 24s",Conv,San Fran,28.58,30.00,30.94,18.50,13.48,17.83,30.00,35.26,25.75,24.66,23.47,26.29,24.12
Artichoke (SF only),Globe,"cartons, 24s",Org,San Fran,37.34,35.46,N\A,N\A,N\A,30.31,35.00,40.12,45.50,40.47,32.76,32.50,36.88
Cabbage,Round green type,"40 lb cartons, medium",Conv,Atlanta,N\A,N\A,N\A,N\A,N\A,N\A,N\A,16.50,N\A,N\A,N\A,N\A,16.50


In [20]:
Veggies2012 = pd.read_csv('Vegetables2012.csv',skiprows=1)

In [21]:
Veggies2013 = pd.read_csv('Vegetables2013.csv', skiprows=1)

In [22]:
#drop the year summary vars
#Veggies2012.columns.get_loc('2012');
Veggies2012 = Veggies2012.drop(Veggies2012.columns[17],1)
Veggies2013 = Veggies2013.drop(Veggies2013.columns[17],1)

In [23]:
#transform the data so dates are a column
Veggies2012 = pd.melt(Veggies2012, id_vars=['Commodity','Subgroup','Package','Organic/ Conventional','Terminal Market'],
       var_name='MonthYear', value_name='Price')

Veggies2013 = pd.melt(Veggies2013, id_vars=['Commodity','Subgroup','Package','Organic/ Conventional','Terminal Market'],
       var_name='MonthYear', value_name='Price')

In [24]:
#put em together
Ags = Veggies2012.append(Veggies2013)

In [25]:
#make sure month year is formatted correctly then convert it to datetime
#Veggies['MonthYear'].str.strip('\n')
Ags['MonthYear'] = Ags['MonthYear'].str.replace('-','20')
Ags['MonthYearForm'] = Ags['MonthYear'].apply(lambda x: parse(x,default=DEFAULT))

In [26]:
#convert price to number formatting
Ags['Price'] = pd.to_numeric(Ags['Price'],errors='coerce')

In [27]:
#want to have it so price for conventional and organic are columns
#couldnt quickly google how-to, so doing it 'by hand'
#first make the two columns, with price if there is one
Ags['Price_Conv'] = np.where(Ags['Organic/ Conventional']=='Conv',Ags['Price'],'NaN')
Ags['Price_Org'] = np.where(Ags['Organic/ Conventional']=='Org',Ags['Price'],'NaN')

In [28]:
#next convert new cols to numbers, my NaNs werent numbers
Ags['Price_Conv'] = pd.to_numeric(Ags['Price_Conv'],errors='coerce')
Ags['Price_Org'] = pd.to_numeric(Ags['Price_Org'],errors='coerce')

In [29]:
#back fill nans so one row has an entry for both cols
Ags['Price_Conv'] = Ags['Price_Conv'].fillna(method='bfill')
Ags['Price_Org'] = Ags['Price_Org'].fillna(method='bfill')

In [30]:
#price data in new cols is repeated and only the first is correct
#so put NaNs back where they are legit
def convOrg(row):
    if (row['Organic/ Conventional'] == 'Conv' and pd.isnull(row['Price'])):
        row['Price_Conv'] = 'NaN'
    if (row['Organic/ Conventional'] == 'Org' and pd.isnull(row['Price'])):
        row['Price_Org'] = 'NaN'
    return row

In [31]:
Ags = Ags.apply(convOrg,axis=1)

In [32]:
#drop now unwanted columns
#Ags.columns.get_loc('Organic/ Conventional');
Ags = Ags.drop(Ags.columns[[3,5,6]],1)

In [33]:
#drop dupes keep first entry bc thats the good one
Ags = Ags.drop_duplicates(['Commodity','MonthYearForm','Subgroup','Package','Terminal Market'], keep='first')

In [34]:
#next convert new cols to numbers, my NaNs werent numbers
Ags['Price_Conv'] = pd.to_numeric(Ags['Price_Conv'],errors='coerce')
Ags['Price_Org'] = pd.to_numeric(Ags['Price_Org'],errors='coerce')

In [35]:
#calculate the organic premium
Ags['Premium'] = Ags['Price_Org'] - Ags['Price_Conv']
Ags['PremiumAbs'] = (Ags['Price_Org'] - Ags['Price_Conv'])

In [36]:
Ags = Ags.set_index(['MonthYearForm'])

In [37]:
#create groups to analyze
gCSPT0 = Ags.groupby(['Commodity','Subgroup','Package','Terminal Market'])

In [236]:
from bokeh.models import ColumnDataSource, HoverTool, Legend, Label
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Select
from bokeh.io import output_notebook, show, vform, output_file
from bokeh.plotting import figure
from bokeh.layouts import column

output_notebook()

Loading BokehJS ...

In [39]:
#subset for simplicity
veglist =[]
for i, group in gCSPT0:
    veglist.append(i)

In [40]:
cab = Ags[(Ags.Commodity == 'Cabbage') &
      (Ags.Subgroup == 'Round green type') &
      (Ags.Package == '45 lb cartons, medium') &
      (Ags['Terminal Market'] == 'Atlanta')]

In [185]:
#stats to display below the graph
cabCorr = cab.Price_Conv.corr(cab.Price_Org).round(decimals=2)
cabPremAvg = round(cab.PremiumAbs.mean(),2)

In [174]:
s= "/".join(veglist[1])

In [230]:
p = figure(x_axis_type='datetime', tools='hover,pan,crosshair,box_zoom,reset,save',
          title=s)

p.plot_width = 800
p.plot_height = 500

source = ColumnDataSource(
    data=dict(
        
    #date for x axis
    Date=[x for x in cab.index],
        
    #data for line
    Org=[x for x in cab.Price_Org],
    Conv=[x for x in cab.Price_Conv],
        
    #for the hover tool
    OrgH = cab.Price_Org,
    ConvH = cab.Price_Conv,
    premH = cab.Premium     
    )
)

p.select_one(HoverTool).tooltips = [
    ("Organic: ", "@OrgH"),
    ("Conventional: ", "@ConvH"),
    ("Org. Premium: ", "@premH"),
    
]

p1 = p.line(x="Date", y="Org", legend="Organics", line_color="#a6cee3",source=source)
p2 = p.line(x="Date", y="Conv", legend="Conventional", line_color="#fdbf6f",source=source)

p.legend.orientation = "horizontal"
p.legend.location="top_left"


In [231]:
# w = figure()

# sourcew = ColumnDataSource(
#     data=dict(
    
#     #x and y axes
#     xw = [1,2,3,4],
#     yw = [1]
    
#     ))

# w.plot_height=125
# w.plot_width=800
# w.toolbar_location = None
# w.grid=None
# w.background_fill_color = "white"

In [240]:
show(p)
print("HI")

HI


In [238]:
output_file("test.html")
show(p)

In [97]:
Ags.head(n=10)

,Commodity,Subgroup,Package,Terminal Market,Price_Conv,Price_Org,Premium
MonthYearForm,,,,,,,
2012-01-01,Artichoke (SF only),Globe,"cartons, 24s",San Fran,28.58,37.34,8.76
2012-01-01,Cabbage,Round green type,"40 lb cartons, medium",Atlanta,NaN,33.95,NaN
2012-01-01,Cabbage,Round green type,"45 lb cartons, medium",Atlanta,14.85,33.95,19.10
2012-01-01,Cabbage,Round green type,"45 lb cartons, 18-24s",San Fran,14.85,24.97,10.12
2012-01-01,Carrots,Not specified,25 lb sacks loose,Atlanta,12.50,24.97,12.47
2012-01-01,Carrots,Topped,25 lb sacks loose,San Fran,6.60,22.00,15.40
2012-01-01,Cauliflower,White,"cartons, film wrapped, 12s",Atlanta,15.03,20.10,5.07
2012-01-01,Cauliflower,White,"cartons, film wrapped, 12s",San Fran,11.24,20.10,8.86
2012-01-01,Greens,"Swiss Chard, mixed colors","cartons bunched, 24s",Atlanta,25.03,23.81,-1.22


In [37]:
Ags.head()

,Commodity,Subgroup,Package,Terminal Market,MonthYearForm,Price_Conv,Price_Org,Premium
0,Artichoke (SF only),Globe,"cartons, 24s",San Fran,2012-01-01,28.58,37.34,8.76
2,Cabbage,Round green type,"40 lb cartons, medium",Atlanta,2012-01-01,NaN,33.95,NaN
3,Cabbage,Round green type,"45 lb cartons, medium",Atlanta,2012-01-01,14.85,33.95,19.10
4,Cabbage,Round green type,"45 lb cartons, 18-24s",San Fran,2012-01-01,14.85,24.97,10.12
6,Carrots,Not specified,25 lb sacks loose,Atlanta,2012-01-01,12.50,24.97,12.47


In [418]:
Ags1 = Ags.pivot_table('Price',['Commodity','MonthYearForm','Terminal Market','Subgroup'],'Organic/ Conventional')

In [419]:
Ags1.head()

Organic/ Conventional                                        Conv    Org
Commodity           MonthYearForm Terminal Market Subgroup              
Artichoke (SF only) 2012-01-01    San Fran        Globe     28.58  37.34
                    2012-02-01    San Fran        Globe     30.00  35.46
                    2012-03-01    San Fran        Globe     30.94    NaN
                    2012-04-01    San Fran        Globe     18.50    NaN
                    2012-05-01    San Fran        Globe     13.48    NaN

In [355]:
Ags1.head()

Commodity            MonthYearForm  Terminal Market  Subgroup
Artichoke (SF only)  2012-01-01     San Fran         Globe       28.58
                     2012-02-01     San Fran         Globe       30.00
                     2012-03-01     San Fran         Globe       30.94
                     2012-04-01     San Fran         Globe       18.50
                     2012-05-01     San Fran         Globe       13.48
Name: Conv, dtype: float64

In [323]:
Ags.head()
#Veggies2013.head()

Organic/ Conventional                               Conv    Org
Commodity           MonthYearForm Terminal Market              
Artichoke (SF only) 2012-01-01    San Fran         28.58  37.34
                    2012-02-01    San Fran         30.00  35.46
                    2012-03-01    San Fran         30.94    NaN
                    2012-04-01    San Fran         18.50    NaN
                    2012-05-01    San Fran         13.48    NaN

In [226]:
Veggies.dtypes

Commodity                        object
Subgroup                         object
Package                          object
Organic/ Conventional            object
Terminal Market                  object
MonthYear                        object
Price                            object
MonthYearForm            datetime64[ns]
dtype: object